# 実行環境の作成

## GCP認証

下記コードでGCPに接続

In [1]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-japan
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-japan statistics-japan

## ライブラリ

In [15]:
import pandas as pd
import plotly.express as px
# Google Colab.でプロットするためには，以下を実行する．
import plotly.io as pio
pio.renderers.default = "colab"

# GBQからデータ取得

## テーブルデータ取得

In [5]:
from google.cloud import bigquery

def get_estat_origin(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'estat_ranking'
  client = bigquery.Client(project=project_id)

  query_string = f'SELECT * FROM {dataset_id}.{table_id}'

  return client.query(query_string).result().to_dataframe()

In [6]:
table_id = '0000010101_A1101'
get_estat_origin(table_id).head()

,index,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,45,A1101,01000,1975100000,5338206,人,総人口,北海道,1975年度,5
1,46,A1101,01000,1976100000,5395000,人,総人口,北海道,1976年度,5
2,47,A1101,01000,1977100000,5443000,人,総人口,北海道,1977年度,5
3,48,A1101,01000,1978100000,5490000,人,総人口,北海道,1978年度,5
4,49,A1101,01000,1979100000,5535000,人,総人口,北海道,1979年度,5


## テーブルデータを結合

In [7]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
    ]
}

In [50]:
import pandas as pd

def df_data(card):
  
  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 結合
  for c in card['categories']:
    df = get_estat_origin(list(c.values())[0])
    df['categoryName'] = list(c.keys())[0]
    df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
    df['rankJapan'] = df.astype(str).apply(lambda x: '全国 第' + x['rankJapan'] + '位', 1)
    df = df.drop('index', axis=1)

    df_res = pd.concat([df_res, df])
  
  return df_res

In [44]:
df_data(card).head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,A1101,01000,1975,5338206,人,総数,北海道,1975年度,全国 第5位
1,A1101,01000,1976,5395000,人,総数,北海道,1976年度,全国 第5位
2,A1101,01000,1977,5443000,人,総数,北海道,1977年度,全国 第5位
3,A1101,01000,1978,5490000,人,総数,北海道,1978年度,全国 第5位
4,A1101,01000,1979,5535000,人,総数,北海道,1979年度,全国 第5位


## 地域ごとのJSONに保存

In [53]:
import tqdm

def save_json(card):

  df = df_data(card)
  
  areas = sorted(list(set(df['areaCode'].tolist())))
  for area in tqdm.notebook.tqdm(areas):
    df_area = df.copy().query(f'areaCode == "{area}"').reset_index()
    df_area = df_area[['categoryName', 'timeCode', 'timeName', 'value', 'unit', 'rankJapan']]

    gcs_path = "statistics-japan/prefecture/{}_{}.json".format(card['cardId'],area)
    df_area.to_json(gcs_path, orient='records', force_ascii=False)
    
  return df.query(f'areaCode == "28000"').reset_index().head()

In [40]:
save_json(card)

  0%|          | 0/47 [00:00<?, ?it/s]

,categoryName,timeCode,timeName,value,unit,rankJapan
0,総数,1975,1975年度,1042572,人,38
1,総数,1976,1976年度,1058000,人,38
2,総数,1977,1977年度,1071000,人,38
3,総数,1978,1978年度,1081000,人,38
4,総数,1979,1979年度,1094000,人,37
...,...,...,...,...,...,...
130,女性,2015,2015年度,728947,人,28
131,女性,2016,2016年度,732000,人,26
132,女性,2017,2017年度,734000,人,25
133,女性,2018,2018年度,736000,人,25


# 人口・世帯

### 総人口

In [16]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
    ]
}

In [49]:
save_json(card)

  0%|          | 0/47 [00:00<?, ?it/s]

## 年齢調整死亡率

In [ ]:
card = {
    'cardId': 'mortality',
    'categories': [
        { '男性' : '0000010101_A424001' },
        { '女性' : '0000010101_A424002' },
    ]
}
save_json(card)

## 年齢中位数

In [54]:
card = {
    'cardId': 'median-age',
    'categories': [
        { '年齢中位数' : '0000010101_A1231' },
    ]
}
save_json(card)

  0%|          | 0/47 [00:00<?, ?it/s]

,index,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,27,A1231,28000,2010,45.0,歳,年齢中位数,兵庫県,2010年度,全国 第36位
1,74,A1231,28000,2015,47.0,歳,年齢中位数,兵庫県,2015年度,全国 第34位


## 年齢別人口

In [57]:
card = {
    'cardId': 'median-age',
    'categories': [
        { "0～4歳人口（男）": "0000010101_A120101"}, 
        { "0～4歳人口（女）": "0000010101_A120102"}, 
        { "5～9歳人口（男）": "0000010101_A120201"}, 
        { "5～9歳人口（女）": "0000010101_A120202"}, 
        { "10～14歳人口（男）": "0000010101_A120301"}, 
        { "10～14歳人口（女）": "0000010101_A120302"}, 
        { "15～19歳人口（男）": "0000010101_A120401"}, 
        { "15～19歳人口（女）": "0000010101_A120402"},  
        { "20～24歳人口（男）": "0000010101_A120501"}, 
        { "20～24歳人口（女）": "0000010101_A120502"}, 
        { "25～29歳人口（男）": "0000010101_A120601"}, 
        { "25～29歳人口（女）": "0000010101_A120602"}, 
        { "30～34歳人口（男）": "0000010101_A120701"}, 
        { "30～34歳人口（女）": "0000010101_A120702"}, 
        { "35～39歳人口（男）": "0000010101_A120801"}, 
        { "35～39歳人口（女）": "0000010101_A120802"}, 
        { "40～44歳人口（男）": "0000010101_A120901"}, 
        { "40～44歳人口（女）": "0000010101_A120902"}, 
        { "45～49歳人口（男）": "0000010101_A121001"}, 
        { "45～49歳人口（女）": "0000010101_A121002"}, 
        { "50～54歳人口（男）": "0000010101_A121101"}, 
        { "50～54歳人口（女）": "0000010101_A121102"}, 
        { "55～59歳人口（男）": "0000010101_A121201"}, 
        { "55～59歳人口（女）": "0000010101_A121202"}, 
        { "60～64歳人口（男）": "0000010101_A121301"}, 
        { "60～64歳人口（女）": "0000010101_A121302"}, 
        { "65～69歳人口（男）": "0000010101_A121401"}, 
        { "65～69歳人口（女）": "0000010101_A121402"}, 
        { "70～74歳人口（男）": "0000010101_A121501"}, 
        { "70～74歳人口（女）": "0000010101_A121502"}, 
        { "75～79歳人口（男）": "0000010101_A121601"}, 
        { "75～79歳人口（女）": "0000010101_A121602"}, 
        { "80～84歳人口（男）": "0000010101_A121701"}, 
        { "80～84歳人口（女）": "0000010101_A121702"}, 
        { "85～89歳人口（男）": "0000010101_A121801"}, 
        { "85～89歳人口（女）": "0000010101_A121802"}, 
        { "90～94歳人口（男）": "0000010101_A121901"}, 
        { "90～94歳人口（女）": "0000010101_A121902"}, 
        { "95～99歳人口（男）": "0000010101_A122001"}, 
        { "95～99歳人口（女）": "0000010101_A122002"}, 
        { "100歳以上人口（男）": "0000010101_A122101"}, 
        { "100歳以上人口（女）": "0000010101_A122102"}
    ]
}
# save_json(card)

In [58]:
df = df_data(card)
df

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,A120101,13000,1975,494161,人,0～4歳人口（男）,東京都,1975年度,全国 第1位
1,A120101,13000,1976,476000,人,0～4歳人口（男）,東京都,1976年度,全国 第1位
2,A120101,13000,1977,451000,人,0～4歳人口（男）,東京都,1977年度,全国 第1位
3,A120101,13000,1978,423000,人,0～4歳人口（男）,東京都,1978年度,全国 第1位
4,A120101,13000,1979,394000,人,0～4歳人口（男）,東京都,1979年度,全国 第1位
...,...,...,...,...,...,...,...,...,...
371,A122102,43000,2015,1169,人,100歳以上人口（女）,熊本県,2015年度,全国 第15位
372,A122102,44000,2015,652,人,100歳以上人口（女）,大分県,2015年度,全国 第30位
373,A122102,45000,2015,691,人,100歳以上人口（女）,宮崎県,2015年度,全国 第29位
374,A122102,46000,2015,1141,人,100歳以上人口（女）,鹿児島県,2015年度,全国 第16位


In [173]:

def pyramid(df):
  categories = list_categories(df)

  times = list_times(df)

  
  # 年次と地域で抽出
  df_time = df[df.copy()['timeCode'] == '2015']

  df_area = df_time[df_time['areaCode'] == '28000']
  # print(df_area)

 

  man = data_man(df_area)
  # print(man)

  woman = data_woman(df_area)
  # print(woman)

  # print(test(df))

  # for category in categories:
  #   df_man = df[df.copy()['categoryName'] == category+'（男）'][['timeCode','value']].rename(columns={'value': '男性'})
  #   df_man['categoryName'] = category

  #   df_woman = df[df.copy()['categoryName'] == category+'（女）'][['timeCode','value']].rename(columns={'value': '女性'})
  #   df_woman['categoryName'] = category
  #   df_woman['女性'] = df_woman['女性']*-1

  #   df_c = pd.merge(df_man, df_woman,on=['timeCode','categoryName'], how='outer')
  #   print(df_c)


  # return {
  #     'catgories': categories,
  #     'times': times
  # }
  return

# def test(df):
#   return [
#       {'name': '男性', 'data': [] },
#       {'name': '女性', 'data': [] },
#   ]

def data_man(df):
  df = df.query('categoryName.str.contains("男")')
  return df['value'].tolist()

def data_woman(df):
  df = df.query('categoryName.str.contains("女")', engine='python')
  return list(map(lambda x: x*-1, df['value'].tolist()))

def list_categories(df):
  return df['categoryName'].str.replace('（女）', '').str.replace('（男）', '').drop_duplicates().reset_index()['categoryName'].tolist()

def list_times(df):
  return df[['timeCode','timeName']].drop_duplicates(subset=['timeCode']).sort_values('timeCode').to_dict(orient='records')



In [174]:
pyramid(df)

TypeError: ignored